# Consignador RNN

## Contenido
1. [Importar librerias](#Importar-librerias)
2. [Cargar datos](#Cargar-datos)
3. [Preparar datos](#Preparar-datos)
4. [Modelo](#Modelo)
5. [Entrenamiento](#Entrenamiento) (#TODO: agregar grafico de perdida y guardar mejor modelo en cada epoca)
6. [Validacion](#Validacion)
7. [Prediccion](#Prediccion)
8. [Guardar modelo](#Guardar-modelo)


## Importar librerias

In [ ]:
# Importar librerias
import utilities as ut
import pandas as pd
import numpy as np
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from keras.models import load_model, Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.metrics import AUC, Precision, Recall
from matplotlib import pyplot as plt

## Cargar datos

In [ ]:
data = pd.read_pickle(ut.INCIDENTES_CAMARAS_FILENAME)
# convertir el id_camara a int
data['id_camara'] = data['id_camara'].astype(int)
data

#### Cargar datos de puntos de interes

In [ ]:
DIR_PATH = '\\\\C4wadpninv004\\ANALISIS II-DGGE\\02. SME\\GUSTAVO\\'
# DIR_PATH = '..\\data\\'
FILE_PATH = DIR_PATH + 'B200m_CONSIGNADAS.csv'
FILE_PATH = DIR_PATH + 'B200m_CONSIGNADAS-v3.csv'
data_interest_points = pd.read_csv(FILE_PATH, sep=',', encoding='latin-1')
# Renombrar la columna 'Etiquetas' a 'id_camara'
data_interest_points.rename(columns={'id': 'id_camara'}, inplace=True)
# Ordenar los datos por la columna 'id_camara'
data_interest_points.sort_values(by=['id_camara'], inplace=True)
# Resetear los índices
data_interest_points.reset_index(drop=True, inplace=True)

data_interest_points

#### Agrupar los puntos de interes

In [ ]:
IP_groups = [['C09-CUARTELES DE LA POLICÍA AUXILIAR','C10-CUARTELES PBI','C11-CUARTELES PGJ','C16-JUZGADOS CIVILES Y PENALES','C20-MINISTERIOR PUBLICOS','C21-MODULOS SSP',],    # P01 - Seguridad

['C05-CENTRALES CAMIONERAS','C19-METROBUS','C31-TREN LIGERO','C42-TROLEBUS','C43-TURIBUS','C72-ACCESOS METRO','C81-ESTACIONES DE CABLEBUS',],                                        # P02 - Transporte

['C06-CENTROS COMERCIALES','C38-MERCADOS PUBLICOS','C49-TIENDAS DEPARTAMENTALES','C53-CINES','C62-OXXO',],#'C46-ESTABLECIMIENTOS MERCANTILES',],                                    # P03 - Comercio

['C22-MONUMENTOS HISTORICOS', 'C57-EVENTOS MASIVOS', 'C73-ATRACTIVOS TURISTICOS', 'C78-ZONAS ARQUEOLOGICAS',],                                                                        # P04 - Turismo

['C58-CASAS Y CENTROS DE CULTURA','C59-MUSEOS Y TEATROS','C75-FONOTECAS','C76-FOTOTECA','C77-GALERIAS', 'C26-PLAZAS Y PARQUES',],                                                    # P05 - Cultura

['C23-NOTARIAS','C24-OFICINAS DE GOBIERNO','C27-RECLUSORIOS','C30-TESORERIAS','C82-ALCALDIAS', 'C01-BANCOS'],                                                                        # P06 - Gobierno

['C15-HOSPITALES','C45-CENTROS DE SALUD Y CLINICAS',],                                                                                                                                # P07 - Hospitales

['C44-GUARDERIAS', 'C67-CENTROS PILARES', 'C13-EDUCACION',],#'C13-CAM', 'C13-CPARA EL TRABAJO', 'C13_PREESC', 'C13_PRIM', 'C13_SEC', 'C13_BACH', 'C13_SUP',],                                            # P08 - Escuelas

['C33-IGLESIAS Y TEMPLOS',],                                                                                                                                                        # P09 - Iglesias

['C51-EDIFICIOS',],]                                                                                                                                    # P10 - Edificios

# Cada nueva columna es la suma de las columnas que se encuentran en la lista de agrupaciones con nombre GRUPO-k
for i in range(len(IP_groups)):
    data_interest_points['P.Interes-'+str(i+1)] = data_interest_points[IP_groups[i]].sum(axis=1)
    # # Eliminamos las columnas que ya no se van a utilizar
    data_interest_points = data_interest_points.drop(IP_groups[i], axis=1)
data_interest_points = data_interest_points.drop(["C46-ESTABLECIMIENTOS MERCANTILES"], axis=1)

# Estandarizamos los datos
scaler = StandardScaler(with_mean=False, with_std=False)
data_interest_points[data_interest_points.filter(regex='^P.Interes-\d+').columns] = scaler.fit_transform(data_interest_points[data_interest_points.filter(regex='^P.Interes-\d+').columns])
# data_interest_points[data_interest_points.filter(regex='^P.Interes-\d+').columns] = data_interest_points[data_interest_points.filter(regex='^P.Interes-\d+').columns].copy()
data_interest_points

In [ ]:
data['date'] = data['fecha_creacion'].dt.strftime('%Y-%m-%d').astype('datetime64[ns]')
camaras_fechas_frecuencias = data.groupby(['id_camara', 'date']).size().reset_index(name='n_incidentes')

# Convertir la columna de fecha a datetime si aún no lo es
camaras_fechas_frecuencias['date'] = pd.to_datetime(camaras_fechas_frecuencias['date'])

# Crear un DataFrame con todas las fechas posibles

fechas = pd.date_range(start='2021-01-01', end=camaras_fechas_frecuencias['date'].max().date().isoformat())

# Obtener los id únicos de camara
camaras = camaras_fechas_frecuencias['id_camara'].unique()

# Crear un DataFrame con todas las combinaciones posibles de id_camara y fechas
df_total = pd.DataFrame(index=pd.MultiIndex.from_product([camaras, fechas], names=['id_camara', 'date'])).reset_index()

# Unir el DataFrame total con el DataFrame original
data_final = pd.merge(df_total, camaras_fechas_frecuencias, how='left', on=['id_camara', 'date'])

# Llenar los valores nulos con 0
data_final['n_incidentes'] = data_final['n_incidentes'].replace(np.nan, 0)

# Ahora desagregamos la fecha
data_final['anio'] = data_final['date'].dt.year
data_final['mes'] = data_final['date'].dt.month
data_final['dia_semana'] = data_final['date'].dt.dayofweek+1
data_final['dia'] = data_final['date'].dt.day
# data_final['hora'] = data_final['date'].dt.hour

# data_final.drop(['date'], axis=1, inplace=True)



data_final

#### Asignar los puntos de interes a la cada registro de la base de datos de 'data'

In [ ]:
# Recorrer los registros del df data y agregar las columnas de P.Interes de acuerdo a la columna id_camara
interest_points_columns = data_interest_points.filter(regex='^P.Interes-\d+').columns
data_final[data_interest_points.filter(regex='^P.Interes-\d+').columns] = 0
for i in range(len(data_final)):
    print(f"Procesando registro {i+1} de {len(data_final)}", end='\r')
    id_camara = data_final['id_camara'].iloc[i]
    # Buscar el id_camara en el df data_interest_points
    index = data_interest_points[data_interest_points['id_camara'] == id_camara].index
    if len(index) > 0:
        data_final.loc[i, interest_points_columns] = data_interest_points.loc[index[0], interest_points_columns]#**2

# data_final['n_incidentes'] = data_final['n_incidentes'].apply(lambda x: 0 if x == 0 else 1)

data = data_final.copy()
data

In [ ]:

date_split = '2023-01-01'
# date_split = '2022-10-01'

#Split data into train and test, train will be until 2022 and test will be rest of the data
train = data[data['date'] < date_split]
test = data[data['date'] >= date_split]

#Split train and test into X and y
X_train = train.drop(['id_camara', 'n_incidentes', 'date'], axis=1)
y_train = train['n_incidentes']
X_test = test.drop(['id_camara', 'n_incidentes', 'date'], axis=1)
y_test = test['n_incidentes']

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
# Imprimir el porcentaje de train y test respecto al total
print(f"Porcentaje de train: {round(len(train)/len(data)*100, 2)}%")
print(f"Porcentaje de test: {round(len(test)/len(data)*100, 2)}%")

### Oversampling

In [ ]:
# Oversampling para las clases minoritarias
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

# Resumen de la distribución de clases
from collections import Counter
counter = Counter(y_train)
print(counter)


### Modelo de Red Neuronal - Regresion Logistica

##### Deteccion del ultimo modelo ya existente

In [ ]:
MODELS_PATH = '../models/'
import os

# Find last model id from filename structure: model-{id}-{epoch:03d}-{accuracy:.3f}.h5
def find_last_model_id():
    last_model_id = 0
    for file in os.listdir(MODELS_PATH):
        if file.startswith('model-'):
            model_id = int(file.split('-')[1])
            if model_id > last_model_id:
                last_model_id = model_id
    return last_model_id
print(find_last_model_id())


##### Creacion del modelo

In [ ]:
# Creacion del modelo con grafica de perdida y accuracy
# Pasos:
# 1. Definir el modelo
# 2. Entrenar el modelo
# 3. Evaluar el modelo
# 4. Guardar el modelo
# 5. Cargar el modelo
# 6. Hacer predicciones con el modelo

# Solo toma como el total, los que son 1 y de esos verifica cuantos y_pred son 1
def c5_score(y_true, y_pred):
	import tensorflow as tf
	y_true = tf.cast(y_true, tf.float32) # cast means convert
	y_pred = tf.cast(tf.round(y_pred), tf.float32) # round means round to nearest integer
	# probando el valor que debera regresar
	y_pred[y_true != 0] = 1
	return tf.keras.metrics.binary_accuracy(y_true, y_pred)

# 1 Definir el modelo
def create_model(input_dim, output_dim, hidden_layers, neurons):
    model = Sequential()
    model.add(Dense(neurons, input_dim=input_dim, activation='relu'))
    for i in range(hidden_layers):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(output_dim, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adagrad', metrics=['accuracy', c5_score ])
    return model

# 2 Entrenar el modelo
def train_model(model, X_train, y_train, epochs, batch_size, validation_split, model_id):
    # Crear el callback para guardar el modelo
    checkpoint = ModelCheckpoint(MODELS_PATH+'model-'+model_id+'-{accuracy:.4f}-{epoch:03d}.h5', verbose=0, monitor='accuracy', save_best_only=True, mode='auto')

    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split, callbacks=[checkpoint])
    return history

# 3 Evaluar el modelo
def evaluate_model(model, X_test, y_test):
    _, accuracy = model.evaluate(X_test, y_test) # loss, accuracy
    return accuracy

# 4 Guardar el mejor modelo (el que tenga el mejor accuracy)
def save_model(model, model_id, accuracy, epoch):
    model.save(f'{MODELS_PATH}model-{model_id}-{epoch:03d}-{accuracy:.3f}.h5')
    
# 5 Cargar el modelo
def load_model(model_path):
    model = load_model(model_path, custom_objects={'c5_score': c5_score})
    return model

# 6 Hacer predicciones con el modelo
def predict(model, X):
    predictions = model.predict(X)
    return predictions

# 7 Graficar la perdida y el accuracy
def plot_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Precisión del modelo')
    plt.ylabel('Precisión')
    plt.xlabel('Época')
    plt.legend(['Entrenamiento', 'Validación'], loc='upper left')
    plt.show()
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Pérdida del modelo')
    plt.ylabel('Pérdida')
    plt.xlabel('Época')
    plt.legend(['Entrenamiento', 'Validación'], loc='upper left')
    plt.show()
        
# 8 Graficar la matriz de confusión
def plot_confusion_matrix(y_test, y_pred):
    from sklearn.metrics import confusion_matrix
    import seaborn as sns

    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'Blues_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.title('Matriz de confusión')
    plt.show()
        
# 9 Graficar la curva ROC (Receiver Operating Characteristic), es decir, la curva de la sensibilidad (recall) vs la especificidad
def plot_roc_curve(y_test, y_pred):
    from sklearn.metrics import roc_curve

    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.plot(fpr, tpr, marker='.')
    plt.title('Curva ROC')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()


# 1 Definir el modelo
model = create_model(input_dim=X_train.shape[1], output_dim=1, hidden_layers=2, neurons=32)
model.summary()
print("")
model_id = find_last_model_id() + 1

# 2 Entrenar el modelo
history = train_model(model, X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, model_id=str(model_id))

# 3 Evaluar el modelo
accuracy = evaluate_model(model, X_test, y_test)
print(f'Accuracy: {accuracy}')

# 4 Guardar el mejor modelo (el que tenga el mejor accuracy)
save_model(model, model_id, accuracy, len(history.history['accuracy']))

# 5 Cargar el modelo
# model = load_model(f'{MODELS_PATH}model-{model_id}-{len(history.history["accuracy"]):03d}-{accuracy:.3f}.h5')

# 6 Hacer predicciones con el modelo
y_pred = predict(model, X_test)
y_pred = np.round(y_pred).astype(int).reshape(1, -1)[0]
print(y_pred)

# 7 Graficar la perdida y el accuracy
plot_history(history)

# 8 Graficar la matriz de confusión
plot_confusion_matrix(y_test, y_pred)

# 9 Graficar la curva ROC (Receiver Operating Characteristic), es decir, la curva de la sensibilidad (recall) vs la especificidad
plot_roc_curve(y_test, y_pred)

In [ ]:
# # Crear red neuronal de regresión logística
# # Definir el modelo
# model = Sequential()

# # Añadir la capa de entrada y una capa oculta de 64 neuronas
# model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
# # Añadir la capa de entrada y una capa oculta de 32 neuronas
# model.add(Dense(32, activation='relu'))
# # Añadir la capa de entrada y una capa oculta de 32 neuronas
# model.add(Dense(32, activation='relu'))
# # Añadir otra capa oculta
# model.add(Dense(16, activation='relu'))
# # Añadir la capa de salida binaria, 0 o 1, no numeros decimales
# model.add(Dense(1, activation='sigmoid'))

# from keras.optimizers import Adam, Adagrad, SGD

# # Para condicion de paro en convergencia realizar el siguiente paso
# # En la linea 

# # Compilar el modelo
# model.compile(optimizer=Adagrad(learning_rate=0.01),
#               loss='binary_crossentropy', 
#               metrics=['accuracy', AUC(name='auc')])


# # Crear el callback para guardar el modelo
# ID_NEW_MODEL = find_last_model_id() + 1
# checkpoint = ModelCheckpoint(MODELS_PATH+'model-'+str(ID_NEW_MODEL)+'-{accuracy:.4f}-{epoch:03d}.h5', verbose=0, monitor='accuracy', save_best_only=True, mode='auto')
# # Entrenar el modelo
# # model.fit(data.drop(['id_camara','date','n_incidentes'], axis=1), data['n_incidentes'], epochs=100, callbacks=[checkpoint], validation_split=0.2)
# model.fit(X_train, y_train, epochs=100, callbacks=[checkpoint], validation_split=0.2, use_multiprocessing=True, workers=4)

# # Guardar el modelo
# model.save(f'{MODELS_PATH}model-{ID_NEW_MODEL}.h5')

In [ ]:

# Cargar el modelo (Recuerda importar la librería: from keras.models import load_model)
# model = load_model(f'{MODELS_PATH}model-{ID_NEW_MODEL}.h5')
model = load_model(f'{MODELS_PATH}model-2-0.6255-091.h5')
model.evaluate(X_test, y_test) 

In [ ]:
# Predecir
probs = model.predict(X_test)
probs = probs.reshape((probs.shape[0],))
labels = (probs >= 0.5).astype(int)
probs, labels
# np.unique(probs, return_counts=True)
# predictions = np.round(predictions)
# predictions = predictions.astype(int)
# np.unique(labels, return_counts=True)

In [ ]:
# Calcular el accuracy
print("Accuracy:",metrics.classification_report(y_test, labels))

# Calcular la matriz de confusión
print(metrics.confusion_matrix(y_test, labels, labels=y_test.unique()))

In [ ]:
# Realizar dos graficas, una con los datos de labels y otra con los datos de y_test
# El eje x es probs (probabilidades) y el eje y es el redondeo de las probabilidades, el color sera si son verdaderamente 0 o 1
import matplotlib.pyplot as plt
plt.scatter(probs, labels, c=y_test)
plt.show()
